**Experiment for obtaining 24 Hr prediction from Dense Model in rainymotion library**

Author: Divya S. Vidyadharan

File use: For predicting 24 Hr precipitation images. 

Date Created: 19-03-21

Last Updated: 19-03-21

Python version: 3.8.2

In [2]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
from rainymotion.models import Dense

import os
import cv2
import pandas as pd
import wradlib.ipol as ipol # for interpolation
from rainymotion import metrics
from rainymotion import utils
from scipy.ndimage import map_coordinates
import timeit

from matplotlib.cm import get_cmap
os.environ['PROJ_LIB'] = '/anaconda3/pkgs/proj4-5.2.0-h0a44026_1/share/proj/' 
from mpl_toolkits.basemap import Basemap
import imageio

#from tvl1sindysupport import tvl1utilities -in future our own library

In [3]:
#For plotting map - currently using function as in source code Need to change to Cartopy
def plotMap(title,img, lat1, lat2, long1, long2, outputPath,last=0):
   
    (height, width) = img.shape

#     print(img.min(), img.max())
    intensity = np.asarray(img, dtype=np.float32)
#     print(intensity.min(), intensity.max())

    #intensity_level = convert_rep_to_level(intensity).reshape(height, width)
#     print(intensity.min(), intensity.max())
    intensity_level = np.flipud(intensity)

    dLon = (long2 - long1) / width
    dLat = (lat2 - lat1) / height

    lon = np.arange(long1, long2, dLon)
    lat = np.arange(lat1, lat2, dLat)
    lons, lats = np.meshgrid(lon, lat)
#     print(lons.shape, lats.shape)

    fig = plt.figure(figsize=(12, 8))

    # Set up Basemap instance
    m = Basemap(projection="cyl",
                llcrnrlon=long1, urcrnrlon=long2,
                llcrnrlat=lat1, urcrnrlat=lat2,
                resolution='h')

    # Add geographic outlines
    m.drawcoastlines(color='black')
    m.drawstates()
    m.drawcountries()
    m.drawmeridians(np.arange(long1, long2, 1), labels=[True, False, False, True])
    m.drawparallels(np.arange(lat1, lat2, 1), labels=[True, False, True, False])
    #m.drawmeridians(np.arange(new_lon_min, new_lon_max, 1), labels=[False, False, False, False])
    #m.drawparallels(np.arange(new_lat_min, new_lat_max, 1), labels=[False, False, False, False])


    # Plot Data
    #cs = m.contourf(lons, lats, intensity_level, shading='flat', levels=list(range(1, 65)), cmap=get_cmap("jet"))
    
    #cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=get_cmap("gist_earth"))
    cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=discrete_cmap(8,"jet"))
    # Add Colorbar
    if last==1:
        cb = plt.colorbar(cs ,shrink=1.0) #, extend='both')

    # Add Title
    plt.title(title)
    plt.savefig(outputPath, bbox_inches='tight', pad_inches=0.0)
    plt.close()



In [4]:
# For reading data from .h5 files see http://docs.h5py.org/en/stable/quick.html
def readInputData(inputFile):
    initialDataSetNo =60 # The dataset no. to start with
    frames = []
    file = h5py.File(inputFile, 'r')
    datasets = list(file.keys())
    print(len(datasets)) # There are 178 datasets in this file
    for i in range(3):
        print('The item is',datasets[i+initialDataSetNo])
        dset = file[datasets[i+initialDataSetNo]]
        data=np.asarray(dset.value)
        frames.append(data)
        outFrameName=datasets[i+initialDataSetNo]+'_'+str(i)+'.png'
        matplotlib.image.imsave(outFrameName, frames[i])
    frames = np.stack(frames, axis=0)
    file.close()
    print(frames.shape)
    return frames


In [5]:
# Radar images - For example,to read radar images of Typhoon Faxai 
def readRadarImages(inputFolder, startHr,startMin, timeStep,height,width, noOfImages, fileType):
    
    files = (os.listdir(inputFolder))
    files.sort()
    inputRadarImages = []
    firstImgTime = startHr*100+startMin
    
    listTime = [startHr*100+startMin]
    startTime = startHr
    initialTime = startHr
    startTime = startTime*100+startMin
    for i in range(noOfImages-1):
        if "60" in str(startTime+10):
            startTime = initialTime + 1
            initialTime = startTime
            startTime = startTime*100
            listTime.append((startTime))
        else:
            listTime.append((startTime)+10)
            startTime = startTime+10
        
        
    print(listTime)
    for itemNo in range(noOfImages):
        for fileName in files:
            if str(listTime[itemNo]) in fileName:
                #print(fileName)
                if fileName.endswith(fileType):

                    inputFileName =inputFolder+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 

                    recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    recentFrame = np.reshape(recentFrame,(height,width))
                    recentFrame = recentFrame.astype('float16') 
                    inputRadarImages.append(recentFrame)
    inputRadarImages = np.stack(inputRadarImages, axis=0)
    return inputRadarImages


In [6]:
# Common Initialization
eventName = "TyphoneFaxai"
eventDate ="20190908"
eventNameDate = eventName + "_" + eventDate
# For radar images
inputFolder = "./ForExperiments/Exp1/RadarImages/TyphoonFaxai/For21/"
outputFolder= "./ForExperiments/Exp1/Results/"

height = 781
width = 561
fileType='.bin'
timeStep = 10 # for Japan Radar Data

modelName = "Sparse SD"
startHr = 20
startMin = 30
noOfImages = 3
leadSteps = 12
stepRainyMotion = 5 # 5 minutes
outputFilePath = outputFolder+modelName+'_'
outputFilePath = outputFilePath + eventNameDate

print(outputFilePath)

#Latitude and Longitude of Typhoon Faxai

lat1 = 32.5
lat2 = 39
long1 = 136
long2 = 143



./ForExperiments/Exp1/Results/Sparse SD_TyphoneFaxai_20190908


**1.3 Dense**

In [7]:
modelName = "Dense"
startHr = 20
startMin= 40
noOfImages = 2# Sparse Model needs 24 frames
predStartHr = 2100
step = 5
leadSteps = 12
outputFilePath = outputFolder+'/'+modelName+'/'
outputFilePath = outputFilePath + eventNameDate

print(outputFilePath)



# of_method = "DIS"
# direction = "backward"
# advection = "constant-vector"
# interpolation = "idw"
model = Dense()
model.input_data = readRadarImages(inputFolder, startHr,startMin,timeStep,height,width, noOfImages, fileType)
start = timeit.timeit()
nowcastDense = model.run()
end = timeit.timeit()
sparseTime = end - start
print("Dense took ",end - start)
nowcastDense.shape
print("Saving the nowcast images. Please wait...")
for i in range(leadSteps):
    outFrameName = outputFilePath + '_'+str(predStartHr+(i*5))+'.png'
    #matplotlib.image.imsave(outFrameName, nowcastDense[i])
    if i == leadSteps-1:
        last = 1
    else:
        last = 0
    plotMap(modelName+' '+str(predStartHr+(i*5)),nowcastDense[i], lat1, lat2, long1, long2, outFrameName,last)
print("Finished Dense model nowcasting!")

./ForExperiments/Exp1/Results//Dense/TyphoneFaxai_20190908


FileNotFoundError: [Errno 2] No such file or directory: './ForExperiments/Exp1/RadarImages/TyphoonFaxai/For21/'

In [23]:
import numpy as np
def getGroundTruthImages(recentFramePath,groundTruthTime,height,width,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    groundTruthImages = []
    for fileName in files:
        if fileName.endswith(fileType):
            #if groundTruthTime in fileName:
            #print("The ground truth at %s is available",groundTruthTime)
            inputFileName =recentFramePath+'/'+fileName
            fd = open(inputFileName,'rb')
            #print(inputFileName)

            # straight to numpy data (no buffering) 
            recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
            recentFrame = np.reshape(recentFrame,(height,width))
            recentFrame = recentFrame.astype('float16') 
            
            #print(recentFrame.shape)
            groundTruthImages.append(recentFrame)
            #else:
            #    print("Sorry, unable to find file.")
    groundTruthImages = np.moveaxis(np.dstack(groundTruthImages), -1, 0)
    #print(groundTruthImages.shape)
    return groundTruthImages

**2.1 Mean Absolute Error**

In [24]:
from rainymotion import metrics
def MAE(obs, sim):
    """
    Mean absolute error

    Reference: https://en.wikipedia.org/wiki/Mean_absolute_error

    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations

    Returns:
        float: mean absolute error between observed and simulated values

    """
    obs = obs.flatten()
    sim = sim.flatten()

    return np.mean(np.abs(sim - obs))

def prep_clf(obs, sim, threshold=0.1):

    obs = np.where(obs >= threshold, 1, 0)
    sim = np.where(sim >= threshold, 1, 0)

    # True positive (TP)
    hits = np.sum((obs == 1) & (sim == 1))

    # False negative (FN)
    misses = np.sum((obs == 1) & (sim == 0))

    # False positive (FP)
    falsealarms = np.sum((obs == 0) & (sim == 1))

    # True negative (TN)
    correctnegatives = np.sum((obs == 0) & (sim == 0))

    return hits, misses, falsealarms, correctnegatives


def CSI(obs, sim, threshold=0.1):
    """
    CSI - critical success index

    details in the paper:
    Woo, W., & Wong, W. (2017).
    Operational Application of Optical Flow Techniques to Radar-Based
    Rainfall Nowcasting.
    Atmosphere, 8(3), 48. https://doi.org/10.3390/atmos8030048

    Args:
        obs (numpy.ndarray): observations
        sim (numpy.ndarray): simulations
        threshold (float)  : threshold for rainfall values binaryzation
                             (rain/no rain)

    Returns:
        float: CSI value

    """

    hits, misses, falsealarms, correctnegatives = prep_clf(obs=obs, sim=sim,
                                                           threshold=threshold)

    return hits / (hits + misses + falsealarms)



In [25]:
event_name = "Typhoon Faxai 09 August, 2019"
start = "21:00" 
end = "21:50"
t = ['21:00','21:10','21:20','21:30','21:40', '21:50']

In [26]:
groundTruthPath = "./ForExperiments/Exp1/GroundTruth/TyphoonFaxai"
groundTruthTime = "2100"
groundTruthImgs = getGroundTruthImages(groundTruthPath,groundTruthTime,height,width,fileType)
#print("Ground truth images shape", groundTruthImgs.shape)

MAEDense = []

noOfPrecipitationImages = 6
j = 0  # using another index to skip 5min interval data from rainymotion
for i in range(noOfPrecipitationImages):
    #print(groundTruthImgs[i].shape)
    #print(nowcast[j].shape)
    
    mae = MAE(groundTruthImgs[i],nowcastDense[j])
    MAEDense.append(mae)
   
     
    j = j + 2

    
    

**2.2 Critical Success Index**

In [27]:

CSIDense = []

noOfPrecipitationImages = 6
thres=1.0 #0.1 default
j = 0  # using another index to skip 5min interval data from rainymotion
for i in range(noOfPrecipitationImages):
    #print(groundTruthImgs[i].shape)
    #print(nowcast[j].shape)
    
    csi = CSI(groundTruthImgs[i],nowcastDense[j],thres)
    CSIDense.append(csi)
    
    j = j + 2

In [28]:

print(MAEDense)


[0.8432216456238436, 1.0315296638748273, 1.19831056783877, 1.3073393090805663, 1.4043424869214347, 1.5323481270934254]
[0.9175807609646979, 1.0860179288997813, 1.2283159340174261, 1.3104286818499766, 1.408336839459327, 1.5012389662619305]
[0.719, 1.1045, 1.37, 1.491, 1.665, 1.865]
[0.719, 1.067, 1.312, 1.407, 1.496, 1.62]
[0.967, 1.19, 1.342, 1.448, 1.557, 1.634]
[0.7715, 0.984, 1.151, 1.256, 1.323, 1.438]
[0.967, 1.19, 1.342, 1.448, 1.557, 1.634]
[0.967, 1.19, 1.342, 1.448, 1.557, 1.634]
[0.793, 0.978, 1.145, 1.218, 1.269, 1.344]


In [29]:

print(CSIDense)


[0.7904912102697575, 0.7298682977670629, 0.6966545698514837, 0.6750793883433485, 0.6450264801422202, 0.6182539261928737]
[0.77753947498003, 0.7256793663084538, 0.6971816645284786, 0.6822948245186813, 0.6589511544262675, 0.6392634805995975]
[0.8384256631159492, 0.7474935785897755, 0.706336344052019, 0.6850744441409965, 0.6474018057494213, 0.6226589268487409]
[0.8384256631159492, 0.7479924076507519, 0.7056250942542603, 0.6911027568922306, 0.6622325537264576, 0.6395513217344704]
[0.7738384854087601, 0.7089091627008043, 0.6730920254158635, 0.6476956325074131, 0.6170731465606945, 0.5963644220098795]

6. TVL1 =  [0.8218938234354869, 0.772092530834789, 0.7487265672818566, 0.7320557100793675, 0.7158446925244988, 0.703617142969844]
[0.7738384854087601, 0.7089091627008043, 0.6730920254158635, 0.6476956325074131, 0.6170731465606945, 0.5963644220098795]
[0.7738384854087601, 0.7089091627008043, 0.6730920254158635, 0.6476956325074131, 0.6170731465606945, 0.5963644220098795]
[0.8152241249612815, 0.76